In [ ]:
"""
Cleaning and Statistical Analysis of Metabolomic by Condition (treatment + timepoint), Treatment, and Timepoint

Metabolomic data with metabolites in rows and samples in columns is cleaned, dropping the header and columns with ortholog 
information to prepare for statistical analysis.

This script performs one-way ANOVA and post-hoc Tukey's HSD tests on positive and negative datasets
(pos_melted_renamed and neg_melted_renamed) to assess whether there are statistically significant 
differences in mass measurements across three categorical variables: Condition, Treatment, and Timepoint.

Steps:
1. Group data by each categorical variable for both positive and negative datasets.
2. Perform one-way ANOVA to test for overall group differences.
3. If significant, perform Tukey's HSD to identify specific group differences.

Outputs:
- F-statistic and p-values from ANOVA
- Tukey's test results for all pairwise comparisons
"""

In [94]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
neg = pd.read_csv('polar_neg_glog_proc.csv')
pos = pd.read_csv('polar_pos_glog_proc.csv')

In [121]:
neg.head()
print(neg['Unnamed: 1'].unique())

[nan 'HMDB'
 'HMDB0000167;HMDB0000719;HMDB0001122;HMDB0003656;HMDB0004041;HMDB0031645;HMDB0060587;HMDB0061148;HMDB0061877'
 'HMDB0001448'
 'HMDB0004072;HMDB0006236;HMDB0013815;HMDB0029636;HMDB0029637;HMDB0029638;HMDB0033910;HMDB0038997;HMDB0062765;HMDB0124943'
 'HMDB0060253'
 'HMDB0000079;HMDB0001525;HMDB0029874;HMDB0031547;HMDB0062558'
 'HMDB0000408;HMDB0000491;HMDB0000695;HMDB0001864;HMDB0006024;HMDB0010717;HMDB0012882;HMDB0031214;HMDB0031216;HMDB0033286;HMDB0033302;HMDB0033698;HMDB0034233;HMDB0041602;HMDB0061873;HMDB0061874;HMDB0061881'
 'HMDB0000026;HMDB0000168;HMDB0011733;HMDB0012265;HMDB0033780'
 'HMDB0034106;HMDB0062164'
 'HMDB0000148;HMDB0002393;HMDB0002931;HMDB0003011;HMDB0003339;HMDB0003609;HMDB0006556;HMDB0033550;HMDB0039426;HMDB0060475;HMDB0062183'
 'HMDB0029389' 'HMDB0031160;HMDB0033053;HMDB0033054'
 'HMDB0002285;HMDB0003320;HMDB0004077;HMDB0031172;HMDB0060289;HMDB0240311'
 'HMDB0000262;HMDB0002024;HMDB0004181;HMDB0029736;HMDB0031852;HMDB0040141;HMDB0040143'
 'HMDB0001424;

In [21]:
pos.head()

,Unnamed: 0,Unnamed: 1,time_point,1H,1H.1,1H.2,1H.3,1H.4,1H.5,1H.6,...,7D.7,7D.8,7D.9,7D.10,7D.11,7D.12,7D.13,7D.14,7D.15,7D.16
0,NaN,NaN,treatment,control,control,control,control,control,control,low,...,low,low,low,low,high,high,high,high,high,high
1,KEGG,HMDB,mz,C01_1H,C02_1H,C03_1H,C04_1H,C05_1H,C06_1H,L01_1H,...,L02_7D,L03_7D,L05_7D,L06_7D,H01_7D,H02_7D,H03_7D,H04_7D,H05_7D,H06_7D
2,NaN,NaN,54.00443,16.52150194,15.7158378,15.24266518,15.92847854,14.39099148,14.15789032,16.21469348,...,15.35737427,15.91878291,15.54268074,15.80879348,15.79330171,15.54863786,15.90076659,15.54934825,15.97799148,15.52435357
3,C14150;C19257,HMDB0168957,58.06521,20.20612624,19.30436009,20.09630823,20.04390003,20.08610176,20.15196424,20.38676024,...,20.93377082,20.83197651,20.94361891,20.90017297,19.90120867,20.73504981,20.85958988,20.5348974,20.79429142,20.68113018
4,NaN,NaN,58.98781,17.42832945,16.74738227,16.36755017,16.85294886,16.1328272,15.94229858,17.45895244,...,16.77182166,18.33640244,17.47433469,15.61157511,19.14781332,18.26491564,18.57043866,17.67009678,17.21153554,17.75230097


In [103]:
#rename columns for treatment condition information
new_column_names_pos = pos_renamed.iloc[1].values  # Get values from row 2
pos_renamed.columns = new_column_names_pos  # Set new column names

new_column_names_neg = neg_renamed.iloc[1].values  # Get values from row 2
neg_renamed.columns = new_column_names_neg  # Set new column names

#delete 2nd column and first two rows
pos_subset = pos_renamed.iloc[2:, 2:].reset_index(drop=True)
neg_subset = neg_renamed.iloc[2:, 2:].reset_index(drop=True)
pos_subset.head()

,mz,C01_1H,C02_1H,C03_1H,C04_1H,C05_1H,C06_1H,L01_1H,L02_1H,L03_1H,...,L02_7D,L03_7D,L05_7D,L06_7D,H01_7D,H02_7D,H03_7D,H04_7D,H05_7D,H06_7D
0,54.00443,16.52150194,15.7158378,15.24266518,15.92847854,14.39099148,14.15789032,16.21469348,15.46671003,15.44109987,...,15.35737427,15.91878291,15.54268074,15.80879348,15.79330171,15.54863786,15.90076659,15.54934825,15.97799148,15.52435357
1,58.06521,20.20612624,19.30436009,20.09630823,20.04390003,20.08610176,20.15196424,20.38676024,20.1693618,20.51398649,...,20.93377082,20.83197651,20.94361891,20.90017297,19.90120867,20.73504981,20.85958988,20.5348974,20.79429142,20.68113018
2,58.98781,17.42832945,16.74738227,16.36755017,16.85294886,16.1328272,15.94229858,17.45895244,16.92306087,16.70132464,...,16.77182166,18.33640244,17.47433469,15.61157511,19.14781332,18.26491564,18.57043866,17.67009678,17.21153554,17.75230097
3,59.06855,15.83662219,15.79341429,14.19066193,15.96319733,14.74727866,14.28540114,14.30260005,15.73895992,14.97676699,...,15.60482298,15.32503756,15.58533299,15.33256625,15.6687864,15.42244725,15.36077386,15.17170028,15.36612361,15.23168776
4,59.99867,15.75953506,15.55858848,15.39640754,16.01870031,15.57303617,15.60661329,15.77800824,15.47588586,15.74722403,...,14.86812001,15.49638082,14.92181152,14.78488454,15.52649239,15.16582467,15.68510631,15.14756844,15.13960483,15.2259729


In [118]:
#this is actually the cleaned dataframe. Save to csv so it can also be used for WGCNA
pos_subset.to_csv('metabolites_pos_cleaned.csv',index=False)
neg_subset.to_csv('metabolites_neg_cleaned.csv',index=False)

In [83]:
# Melt the DataFrame to convert from wide to long format
pos_melted = pd.melt(pos_subset, id_vars=['time_point'], var_name='Condition', value_name='Value')
neg_melted = pd.melt(neg_subset, id_vars=['time_point'], var_name='Condition', value_name='Value')
pos_melted.head()

,mz,Condition,Value
0,54.00443,C01_1H,16.52150194
1,58.06521,C01_1H,20.20612624
2,58.98781,C01_1H,17.42832945
3,59.06855,C01_1H,15.83662219
4,59.99867,C01_1H,15.75953506


In [106]:
#add condition and timepoint information separately
pos_melted['timepoint'] = pos_melted['Condition'].str.split('_').str[1]  # Extracting the part after the '_'
pos_melted['treatment'] = pos_melted['Condition'].str[0]  # Extracting the first letter

neg_melted['timepoint'] = neg_melted['Condition'].str.split('_').str[1]  # Extracting the part after the '_'
neg_melted['treatment'] = neg_melted['Condition'].str[0]  # Extracting the first letter

,mz,Condition,Value,timepoint,treatment
0,74.02460,C01_1H,14.829246,1H,C
1,85.02924,C01_1H,14.127280,1H,C
2,88.04013,C01_1H,14.794095,1H,C
3,88.04066,C01_1H,14.041642,1H,C
4,90.02746,C01_1H,14.160951,1H,C


In [107]:
#change columns to numeric
pos_melted['mz'] = pd.to_numeric(pos_melted['mz'])
neg_melted['mz'] = pd.to_numeric(neg_melted['mz'])
pos_melted['Value'] = pd.to_numeric(pos_melted['Value'])
neg_melted['Value'] = pd.to_numeric(neg_melted['Value'])

#rename columns
pos_melted_renamed = pos_melted.rename(columns = {'mz':'m/z','Value':'mass'})
neg_melted_renamed = neg_melted.rename(columns = {'mz':'m/z','Value':'mass'})
pos_melted_renamed.dtypes

m/z          float64
Condition     object
mass         float64
timepoint     object
treatment     object
dtype: object

In [108]:
#clean potential whitespace
pos_melted_renamed['m/z'] = pos_melted_renamed['m/z'].astype(str).str.strip().astype(float)
neg_melted_renamed['m/z'] = neg_melted_renamed['m/z'].astype(str).str.strip().astype(float)

In [109]:
#perform ANOVA pairwise
# Group data by condition
groups_pos = [group['mass'].values for name, group in pos_melted_renamed.groupby('Condition')]
groups_neg = [group['mass'].values for name, group in neg_melted_renamed.groupby('Condition')]
# Perform ANOVA
f_stat_pos, p_value_pos = stats.f_oneway(*groups_pos)
f_stat_neg, p_value_neg = stats.f_oneway(*groups_neg)

print(f"F-statistic: {f_stat_pos}, p-value: {p_value_pos}")
print(f"F-statistic: {f_stat_neg}, p-value: {p_value_neg}")

F-statistic: 9.490798891034114, p-value: 4.5717554333075426e-212
F-statistic: 7.98104273375402, p-value: 9.938490449797943e-172


In [110]:
#perform ANOVA condition
# Group data by condition
groups_pos = [group['mass'].values for name, group in pos_melted_renamed.groupby('treatment')]
groups_neg = [group['mass'].values for name, group in neg_melted_renamed.groupby('treatment')]
# Perform ANOVA
f_stat_pos, p_value_pos = stats.f_oneway(*groups_pos)
f_stat_neg, p_value_neg = stats.f_oneway(*groups_neg)

print(f"F-statistic: {f_stat_pos}, p-value: {p_value_pos}")
print(f"F-statistic: {f_stat_neg}, p-value: {p_value_neg}")

F-statistic: 16.037449326661758, p-value: 1.0854353924515158e-07
F-statistic: 20.174804229045517, p-value: 1.7332650931822225e-09


In [111]:
#perform ANOVA timepoint
# Group data by condition
groups_pos = [group['mass'].values for name, group in pos_melted_renamed.groupby('timepoint')]
groups_neg = [group['mass'].values for name, group in neg_melted_renamed.groupby('timepoint')]
# Perform ANOVA
f_stat_pos, p_value_pos = stats.f_oneway(*groups_pos)
f_stat_neg, p_value_neg = stats.f_oneway(*groups_neg)

print(f"F-statistic: {f_stat_pos}, p-value: {p_value_pos}")
print(f"F-statistic: {f_stat_neg}, p-value: {p_value_neg}")

F-statistic: 48.554164616471375, p-value: 6.738013444172422e-79
F-statistic: 71.80657078831892, p-value: 9.847102036685594e-119


In [112]:
#TUKEY
tukey_neg_pairwise = pairwise_tukeyhsd(neg_melted_renamed['mass'], neg_melted_renamed['Condition'])
tukey_pos_pairwise = pairwise_tukeyhsd(pos_melted_renamed['mass'], pos_melted_renamed['Condition'])

tukey_neg_condition = pairwise_tukeyhsd(neg_melted_renamed['mass'], neg_melted_renamed['treatment'])
tukey_pos_condition = pairwise_tukeyhsd(pos_melted_renamed['mass'], pos_melted_renamed['treatment'])

tukey_neg_timepoint = pairwise_tukeyhsd(neg_melted_renamed['mass'], neg_melted_renamed['timepoint'])
tukey_pos_timepoint = pairwise_tukeyhsd(pos_melted_renamed['mass'], pos_melted_renamed['timepoint'])

In [113]:
print(tukey_neg_pairwise)
print(tukey_pos_pairwise)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper  reject
------------------------------------------------------
C01_12H  C01_1H  -0.1086    1.0 -0.3955  0.1783  False
C01_12H C01_24H  -0.2882  0.046 -0.5751 -0.0013   True
C01_12H  C01_2H  -0.0311    1.0  -0.318  0.2557  False
C01_12H  C01_4D  -0.0479    1.0 -0.3348   0.239  False
C01_12H  C01_5D  -0.0897    1.0 -0.3765  0.1972  False
C01_12H  C01_6D   -0.244 0.4176 -0.5309  0.0429  False
C01_12H  C01_6H  -0.0421    1.0 -0.3289  0.2448  False
C01_12H  C01_7D  -0.2658 0.1637 -0.5526  0.0211  False
C01_12H C02_12H  -0.0596    1.0 -0.3465  0.2273  False
C01_12H  C02_1H  -0.0849    1.0 -0.3718   0.202  False
C01_12H C02_24H   -0.298 0.0245 -0.5849 -0.0111   True
C01_12H  C02_2H  -0.1313    1.0 -0.4182  0.1555  False
C01_12H  C02_4D  -0.0118    1.0 -0.2987  0.2751  False
C01_12H  C02_5D  -0.0956    1.0 -0.3824  0.1913  False
C01_12H  C02_6D  -0.3472 0.0006 -0.6341 -0.0603   True
C01_12H  C

In [114]:
print(tukey_neg_condition)
print(tukey_pos_condition)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     C      H   0.0389    0.0  0.0184  0.0593   True
     C      L  -0.0151 0.1946 -0.0356  0.0054  False
     H      L   -0.054    0.0 -0.0745 -0.0334   True
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     C      H   0.0134 0.3107 -0.0081  0.0348  False
     C      L  -0.0376 0.0002 -0.0594 -0.0158   True
     H      L   -0.051    0.0 -0.0728 -0.0292   True
----------------------------------------------------


In [115]:
print(tukey_neg_timepoint)
print(tukey_pos_timepoint)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   12H     1H  -0.0351 0.3416 -0.0824  0.0122  False
   12H    24H  -0.2584    0.0 -0.3057 -0.2111   True
   12H     2H  -0.0326 0.4488 -0.0799  0.0147  False
   12H     4D   0.0068    1.0 -0.0405  0.0541  False
   12H     5D  -0.0051    1.0 -0.0525  0.0422  False
   12H     6D  -0.0612  0.002 -0.1085 -0.0139   True
   12H     6H  -0.0091 0.9996 -0.0564  0.0382  False
   12H     7D  -0.1669    0.0 -0.2149 -0.1189   True
    1H    24H  -0.2233    0.0 -0.2699 -0.1767   True
    1H     2H   0.0025    1.0 -0.0441  0.0492  False
    1H     4D   0.0419 0.1194 -0.0047  0.0885  False
    1H     5D     0.03 0.5489 -0.0167  0.0766  False
    1H     6D  -0.0261 0.7244 -0.0727  0.0205  False
    1H     6H    0.026  0.729 -0.0207  0.0726  False
    1H     7D  -0.1318    0.0 -0.1791 -0.0845   True
   24H     2H   0.2258    0.0  0.1792  0.2725 